In [1]:
import sys
sys.path.append('/home/austinwang/austin_big_vision')

import io
import numpy as np
import big_vision.utils as u
import tensorflow as tf

num_lyr = 6
gemma2b_path = '/home/austinwang/gemma2b.npz'
local_np_save_path = '/home/austinwang/gemma2b_first_{num_lyr}.npz'
gcs_np_save_path = f'gs://us-central2-storage/tensorflow_datasets/gemma2b_first_{num_lyr}.npz'

2024-08-23 01:18:37.520178: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-23 01:18:37.542116: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-23 01:18:37.548736: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-23 01:18:38.534408: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
gemma2b = np.load(gemma2b_path)
for k in gemma2b.keys():
    print(k, gemma2b[k].shape, gemma2b[k].dtype)

params/embedder/input_embedding (256128, 2048) float32
params/final_norm/scale (2048,) float32
params/layers/attn/attn_vec_einsum/w (18, 8, 256, 2048) float32
params/layers/attn/kv_einsum/w (18, 2, 1, 2048, 256) float32
params/layers/attn/q_einsum/w (18, 8, 2048, 256) float32
params/layers/mlp/gating_einsum (18, 2, 2048, 16384) float32
params/layers/mlp/linear (18, 16384, 2048) float32
params/layers/pre_attention_norm/scale (18, 2048) float32
params/layers/pre_ffw_norm/scale (18, 2048) float32


In [3]:
gemma2b_half = {}
for k in gemma2b.keys():
    if gemma2b[k].shape[0] == 18:
        gemma2b_half[k] = gemma2b[k][:num_lyr]
    elif k == 'params/final_norm/scale':
        pass
    else:
        gemma2b_half[k] = gemma2b[k]
names_and_vals = [(n,v) for n,v in gemma2b_half.items()]
io_buffer = io.BytesIO()
np.savez(io_buffer, **{k: v for k, v in names_and_vals})

# save to local
with open(local_np_save_path, "wb") as f: f.write(io_buffer.getvalue())
# save to gcs
with tf.io.gfile.GFile(gcs_np_save_path, "wb") as f: f.write(io_buffer.getvalue())

In [4]:
def load_npz(npz_path):
    model_p = u.npload(npz_path)
    for key in model_p.keys():
        print(key, model_p[key].shape, model_p[key].dtype)

load_npz(local_np_save_path)
load_npz(gcs_np_save_path)

params/embedder/input_embedding (256128, 2048) float32
params/layers/attn/attn_vec_einsum/w (6, 8, 256, 2048) float32
params/layers/attn/kv_einsum/w (6, 2, 1, 2048, 256) float32
params/layers/attn/q_einsum/w (6, 8, 2048, 256) float32
params/layers/mlp/gating_einsum (6, 2, 2048, 16384) float32
params/layers/mlp/linear (6, 16384, 2048) float32
params/layers/pre_attention_norm/scale (6, 2048) float32
params/layers/pre_ffw_norm/scale (6, 2048) float32
params/embedder/input_embedding (256128, 2048) float32
params/layers/attn/attn_vec_einsum/w (6, 8, 256, 2048) float32
params/layers/attn/kv_einsum/w (6, 2, 1, 2048, 256) float32
params/layers/attn/q_einsum/w (6, 8, 2048, 256) float32
params/layers/mlp/gating_einsum (6, 2, 2048, 16384) float32
params/layers/mlp/linear (6, 16384, 2048) float32
params/layers/pre_attention_norm/scale (6, 2048) float32
params/layers/pre_ffw_norm/scale (6, 2048) float32
